In [1]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd

In [3]:
years = range(1950, 2017)

In [24]:
dfs = []
for year in years:
    prefix = year >=1958 and 'Billboard_Year-End_Hot_100_singles_of_' \
                or year>1955 and 'Billboard_year-end_top_50_singles_of_' \
                    or 'Billboard_year-end_top_30_singles_of_'
    r = urllib.urlopen('https://en.wikipedia.org/wiki/{}{}'.format(prefix, year)).read()
    soup = BeautifulSoup(r, "lxml")
    table = soup.find('table', class_='wikitable')
    top_songs_of_year = []
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        song_data = [ele for ele in cols if ele]
        if song_data and len(song_data)<3: # after 1983 the list becomes a th instead of a td
            number = int(row.find('th').contents[0])
            song_data = [number] + song_data
        if song_data:
            links = [a['href'] for a in row.find_all('a')]
            link = links and 'https://en.wikipedia.org{}'.format(links[0]) or None
            top_songs_of_year.append([year] + song_data + [link]) # Get rid of empty values
    df = pd.DataFrame(top_songs_of_year)
    df.columns = ['year', 'rank', 'title', 'artist', 'link'] 
    dfs.append(df)



In [183]:
top_100s = pd.concat(dfs)

In [189]:
top_100s = top_100s.reset_index()

In [190]:
top_100s.to_csv('billboard_top_songs.csv', encoding='utf-8')

In [191]:
top_100s.shape

(6181, 6)

In [1]:
def get_aux_data(aux_table):
    aux_data = {}
    data_rows = aux_table.find_all('tr')
    for row in data_rows:
        heading = row.find('th')
        if heading:
            heading = heading.get_text()
            val = None
            if heading in ['Released', 'Length']:
                val = row.find('td').get_text()
                aux_data[heading] = val
            elif heading == 'Genre':
                val = [a.get_text().capitalize() for a in row.find('td').find_all('a') if '#cite' not in a['href']] # genres
                for i, v in enumerate(val, 1):
                    aux_data['{}_{}'.format(heading, i)] = v
            elif heading in ['Songwriter(s)', 'Producer(s)']:
                s = row.find('td').get_text()
                #val = {'{}_{}'.format(heading, i):v for i, v in enumerate([x for x in s.split('\n') if x],1)}
                for i, v in enumerate([x for x in s.split('\n') if x],1):
                    aux_data['{}_{}'.format(heading, i)] = v
    return aux_data


In [193]:
dd = {}
fail_links = {}
for k, v in top_100s.T.to_dict().iteritems():
    print v['year'], v['rank'], v['artist']
    link = v['link']
    try:
        aux_info_link = urllib.urlopen(link).read()
        aux_soup = BeautifulSoup(aux_info_link, "lxml")
        aux_table = aux_soup.find('table', class_='infobox')
        aux_dict = get_aux_data(aux_table)
        z = v.copy()
        z.update(get_aux_data(aux_table))
        dd[k] = z
    except:
        print 'ERROR for: ', v['year'], v['rank'], v['artist']
        fail_links[k] =  link
    
    

    

1950 1 Gordon Jenkins & The Weavers
1950 2 Nat King Cole
1950 3 Anton Karas
1950 4 Gary & Bing Crosby
1950 5 Gary & Bing Crosby
ERROR for:  1950 5 Gary & Bing Crosby
1950 6 Teresa Brewer
1950 7 Guy Lombardo
1950 8 Red Foley
1950 9 Sammy Kaye
1950 10 Sammy Kaye & Don Cornell
1950 11 Eileen Barton
1950 12 Kay Starr
1950 13 Gordon Jenkins & The Weavers
ERROR for:  1950 13 Gordon Jenkins & The Weavers
1950 14 Tony Martin
ERROR for:  1950 14 Tony Martin
1950 15 Phil Harris
ERROR for:  1950 15 Phil Harris
1950 16 Ames Brothers
ERROR for:  1950 16 Ames Brothers
1950 17 Andrews Sisters & Gordon Jenkins
1950 18 Patti Page
1950 19 Andrews Sisters & Gordon Jenkins
1950 20 Tennessee Ernie Ford & Kay Starr
1950 21 Patti Page
1950 22 Gordon Jenkins
ERROR for:  1950 22 Gordon Jenkins
1950 23 Ames Brothers
ERROR for:  1950 23 Ames Brothers
1950 24 Bill Snyder
1950 25 Perry Como
1950 26 Gordon Jenkins
1950 27 Ames Brothers
1950 28 Billy Eckstine
ERROR for:  1950 28 Billy Eckstine
1950 29 Bing Crosby
19

In [194]:
len(fail_links)

208

In [197]:
top_100s_aux = pd.DataFrame(dd).T

In [199]:
top_100s_aux.to_csv('billboard_top_songs_aux.csv', encoding='utf-8')

In [205]:
top_100s_aux.tail()

,Genre_1,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7,Length,Producer(s)_1,Producer(s)_2,...,Songwriter(s)_6,Songwriter(s)_7,Songwriter(s)_8,Songwriter(s)_9,artist,index,link,rank,title,year
6176,Country,NaN,NaN,NaN,NaN,NaN,NaN,4:20,Byron Gallimore,Tim McGraw,...,NaN,NaN,NaN,NaN,Tim McGraw,95,https://en.wikipedia.org/wiki/Humble_and_Kind,96,"""Humble and Kind""",2016
6177,Hip hop,Trap,NaN,NaN,NaN,NaN,NaN,2:53,Metro Boomin,Southside,...,NaN,NaN,NaN,NaN,Future,96,https://en.wikipedia.org/wiki/Wicked_(Future_s...,97,"""Wicked""",2016
6178,Trap,Horrorcore,NaN,NaN,NaN,NaN,NaN,3:59,Dean,Desiigner,...,NaN,NaN,NaN,NaN,Desiigner,97,https://en.wikipedia.org/wiki/Tiimmy_Turner,98,"""Tiimmy Turner""",2016
6179,Hip hop,Pop-rap,NaN,NaN,NaN,NaN,NaN,3:49,DJ Frank E,Charlie Puth,...,NaN,NaN,NaN,NaN,Wiz Khalifa featuring Charlie Puth,98,https://en.wikipedia.org/wiki/See_You_Again_(W...,99,"""See You Again""",2016
6180,Pop,NaN,NaN,NaN,NaN,NaN,NaN,3:50,Julian Bunetta,Shatkin,...,Maureen Anne McDonald,Julian Bunetta,NaN,NaN,One Direction,99,https://en.wikipedia.org/wiki/Perfect_(One_Dir...,100,"""Perfect""",2016


In [223]:
times = map(lambda x:unicode(x).replace('\n', ' ').replace('[1]', '').split(' '), top_100s_aux['Length'].values)

In [232]:
time_list

[u'2:08', u'(see', u'length', u'variations)']

In [233]:
time

In [221]:
filter(lambda x:len(unicode(x).replace('\n', ' '))>4, top_100s_aux['Length'].values)

[u'2:08 (see length variations)',
 u'\n\n\n3:05 (Part 1)\n2:32 (Part 2)\n\n\n',
 u'2:05[1]',
 u'2:31 (Single version)',
 u'2:22 (single version)[1]\n2:55 (album version)[2]',
 u'2-5 minute',
 u'2 minutes 7 seconds',
 u'2:43 (A-side)\n2:48 (B-side)',
 u'\n\n\n3:04 (Detroit version)\n2:50 (National hit version)\n\n\n',
 u'2:32[2]',
 u'3m34s',
 u'2:55 (single version)\n5:28 (album version)',
 u'2:21 (album version)\n3:05 (single version)',
 u'2:15 (part 1)\n2:10 (part 2)',
 u'2:47 (single version)\n3:45 (album version)',
 u'2:33 [1]',
 u'2:49 (part 1), 3:09 (part 2)',
 u'2:14 (45 version)\n02:42 (album cut)',
 u'2:59, 4:29',
 u'\n\n\n1:55 (Part I)\n2:12 (Part II)\n\n\n',
 u'2:48 (Single)\n3:02 (Album)',
 u'3:25 (album)\n3:13 (single)',
 u'3:48 (album)\n3:03 (single)',
 u'3:45 (mono single mix)\n3:22 (stereo album mix)',
 u'3:15, 4:34 (extended version)',
 u'2:18 (stereo version)\n2:26 (mono single version)',
 u'2:47 (original release)\n3:15 (remastered)',
 u'3:01 (with "Good Golly, Miss M